In [6]:
#Code Sample

import requests

def get_google_maps_image(lat, lon, api_key):
    url = f"https://maps.googleapis.com/maps/api/staticmap?center={lat},{lon}&zoom=19&size=400x400&maptype=satellite&key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        filename = f"satellite_image_{lat}_{lon}.png"
        with open(filename, 'wb') as file:
            file.write(response.content)
        return filename
    else:
        return "Error: Image could not be retrieved"

# Example usage
api_key = "" #add your google maps api key here
latitude = 39.762518
longitude = -106.627231
print(get_google_maps_image(latitude, longitude, api_key))


satellite_image_39.762518_-106.627231.png


In [25]:
import pandas as pd
import requests
from google.cloud import storage
import os
from dotenv import load_dotenv

# Get environment variables
load_dotenv()
api_key = os.getenv('GOOGLE_MAPS_API_KEY')
bucket_name = os.getenv('TRAINING_BUCKET_NAME')
os.environ['GOOGLE_CLOUD_PROJECT'] = os.getenv('GOOGLE_CLOUD_PROJECT')

# Set input csv file
input_csv_file = 'usaWild3Month_coords.csv'

# Define a function to upload data to Google Cloud Storage
def upload_to_gcs(bucket_name, data, destination_blob_name):
    """Uploads data to Google Cloud Storage and return the file URL."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_string(data, content_type='image/png')

    public_url = f"https://storage.googleapis.com/{bucket_name}/{destination_blob_name}"
    return public_url

# Define function to get Google Maps image and upload to GCS
def get_google_maps_image(lat, lon, api_key, bucket_name):
    url = f"https://maps.googleapis.com/maps/api/staticmap?center={lat},{lon}&zoom=19&size=400x400&maptype=satellite&key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        destination_blob_name = f"satellite_image_{lat}_{lon}.png"
        return upload_to_gcs(bucket_name, response.content, destination_blob_name)
    else:
        return "Error: You are getting this error from the get_google_maps_image function. Please check the function and try again."

# Read the CSV file using pandas
df = pd.read_csv(input_csv_file)

# Iterate over each row in the dataframe
for index, row in df.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    
    # Call the function with latitude and longitude values
    image_url = get_google_maps_image(latitude, longitude, api_key, bucket_name)

     # Debugging print statements
    print(f"Latitude: {latitude}, Longitude: {longitude}")
    print(f"Image URL: {image_url}")

    if image_url:
        df.at[index, 'maps'] = image_url
    else:
        df.at[index, 'maps'] = "Error you are getting this error from the for loop. Please check the loop and try again."

# Save the updated DataFrame back to CSV
df.to_csv('outputWithMapURL.csv', index=False)


Latitude: 36.201047, Longitude: -114.685437
Image URL: https://storage.googleapis.com/overlandgpt-training-data/satellite_image_36.201047_-114.685437.png
Latitude: 47.61649, Longitude: -122.31823
Image URL: https://storage.googleapis.com/overlandgpt-training-data/satellite_image_47.61649_-122.31823.png
Latitude: 34.01193, Longitude: -113.12459
Image URL: https://storage.googleapis.com/overlandgpt-training-data/satellite_image_34.01193_-113.12459.png
Latitude: 33.67453, Longitude: -115.80265
Image URL: https://storage.googleapis.com/overlandgpt-training-data/satellite_image_33.67453_-115.80265.png
Latitude: 34.17293, Longitude: -116.22466
Image URL: https://storage.googleapis.com/overlandgpt-training-data/satellite_image_34.17293_-116.22466.png
Latitude: 37.207109, Longitude: -112.675489
Image URL: https://storage.googleapis.com/overlandgpt-training-data/satellite_image_37.207109_-112.675489.png
Latitude: 39.610509, Longitude: -112.730474
Image URL: https://storage.googleapis.com/overla